# Импорт библиотек

In [55]:
import numpy as np
import matplotlib.pyplot as plt
import os
import nibabel as nib
import torch
import torchvision.transforms as transforms
import cv2
from PIL import Image
from sklearn.linear_model import LinearRegression
from scipy.ndimage import gaussian_filter
from scipy.signal import convolve
import scipy.signal as signal
import random
from sklearn.metrics import r2_score
import seaborn as sns
import pickle

# Задание окружения

In [14]:
figures = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "figures")

# Работа с данными

## Видеоряд

In [15]:
video_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "src", "Film stimulus.mp4")

In [16]:
def video_to_frames():
    videocap = cv2.VideoCapture(video_path)
    success, frame = videocap.read()
    count = 1
    while success:
        cv2.imwrite(os.path.join(os.path.dirname(os.path.dirname(os.getcwd())),
                                "src", "frames", f"frame_{count}.jpg"), frame)    
        success, frame = videocap.read()
        count += 1

In [17]:
#video_to_frames()

In [18]:
model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet152', pretrained=True, verbose=False)

for param in model.parameters():
    param.requires_grad = False

model.fc = torch.nn.Linear(2048, 2048)

c:\Users\kisnikser\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
c:\Users\kisnikser\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [19]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [20]:
def frames_to_tensors():
    for i in range(1, 9751):
        frame_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "src", "frames", f"frame_{i}.jpg")
        frame = Image.open(frame_path)
        frame_tensor = preprocess(frame)
        frame_tensor = frame_tensor.unsqueeze(0)
        yield frame_tensor

In [21]:
def tensors_to_vectors():
    for frame_tensor in frames_to_tensors():
        # передача картинки в модель и получение выходных данных
        with torch.no_grad():
            output = model(frame_tensor)
        # преобразование выходных данных в вектор
        vector = output.numpy().flatten()
        yield vector

In [22]:
#vector_list = [vector for vector in tensors_to_vectors()]
#np.save("vector_list", vector_list)
vector_list = np.load(os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "src", "vector_list.npy"))

## Снимки фМРТ

# Задание окружения

In [23]:
figures = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "figures")

# Работа с данными

## Видеоряд

In [24]:
video_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "src", "Film stimulus.mp4")

In [25]:
def video_to_frames():
    videocap = cv2.VideoCapture(video_path)
    success, frame = videocap.read()
    count = 1
    while success:
        cv2.imwrite(os.path.join(os.path.dirname(os.path.dirname(os.getcwd())),
                                "src", "frames", f"frame_{count}.jpg"), frame)    
        success, frame = videocap.read()
        count += 1

In [26]:
#video_to_frames()

In [27]:
model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet152', pretrained=True, verbose=False)

for param in model.parameters():
    param.requires_grad = False

model.fc = torch.nn.Linear(2048, 2048)

In [28]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [29]:
def frames_to_tensors():
    for i in range(1, 9751):
        frame_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "src", "frames", f"frame_{i}.jpg")
        frame = Image.open(frame_path)
        frame_tensor = preprocess(frame)
        frame_tensor = frame_tensor.unsqueeze(0)
        yield frame_tensor

In [30]:
def tensors_to_vectors():
    for frame_tensor in frames_to_tensors():
        # передача картинки в модель и получение выходных данных
        with torch.no_grad():
            output = model(frame_tensor)
        # преобразование выходных данных в вектор
        vector = output.numpy().flatten()
        yield vector

In [31]:
#vector_list = [vector for vector in tensors_to_vectors()]
#np.save("vector_list", vector_list)
vector_list = np.load(os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "src", "vector_list.npy"))

## Снимки фМРТ

In [ ]:
subs_with_fmri = ['04', '07', '08', '09', '11', '13', '14', '15', '16', '18',\
                  '22', '24', '27', '28', '29', '31', '35', '41', '43', '44',\
                  '45', '46', '47', '51', '52', '53', '55', '56', '60', '62']

In [43]:
class Sub:

    def __init__(self, number):
        self.number = number
        self.path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "src", "ds003688-download", f"sub-{self.number}",\
                                    "ses-mri3t", "func", f"sub-{self.number}_ses-mri3t_task-film_run-1_bold.nii.gz")
        self.scan = nib.load(self.path)
        self.data = self.scan.get_fdata()
        self.tensor = torch.tensor(self.data)
        self.tensor_np = self.tensor.numpy()

In [44]:
sub = Sub('04')

In [45]:
sub.tensor.shape

torch.Size([40, 64, 64, 641])

# Построение линейной модели

In [136]:
class Preprocessor:

    def __init__(self, sub, dt, coef, train_size=0.7):
        self.sub = sub
        self.dt = dt
        self.coef = coef
        self.train_size = train_size

        self.nu = 25 # частота видео
        self.mu = 641. / 390. # частота снимков фМРТ
        self.d1 = self.sub.tensor.shape[0] # размерности снимка фМРТ до сжатия
        self.d2 = self.sub.tensor.shape[1]
        self.d3 = self.sub.tensor.shape[2]
        self.d4 = self.sub.tensor.shape[3]
        self.d = 2048 # длина вектора признакового описания изображения
        self.N = 641 - int(self.mu * self.dt) # N - количество снимков fMRI

        self.train, self.test = self.get_train_test()
        self.X_train, self.Y_train, self.X_test, self.Y_test = self.get_XY()

    @staticmethod
    def preprocess(v):
        return (v - v.min()) / (v.max() - v.min())

    @staticmethod
    def MSE(A):
        m, n = A.shape
        return 1 / n * np.linalg.norm(A, "fro") ** 2 # усреднение по столбцам матрицы

    def get_train_test(self):
        pairs = [(int(i * self.nu / self.mu), int(self.mu * self.dt + i)) for i in range(self.N)] # (номер изображения, номер снимка)

        if (self.coef > 1): # сжатие снимка фМРТ
            maxpool = torch.nn.MaxPool3d(kernel_size=self.coef, stride=self.coef)
            input_tensor = self.sub.tensor.permute(3, 0, 1, 2)
            output_tensor = maxpool(input_tensor).permute(1, 2, 3, 0)
            self.sub._tensor = output_tensor
            self._d1 = self.sub._tensor.shape[0]
            self._d2 = self.sub._tensor.shape[1]
            self._d3 = self.sub._tensor.shape[2]
            self._d4 = self.sub._tensor.shape[3]
        else:
            self.sub._tensor = self.sub.tensor
            self._d1 = self.sub._tensor.shape[0]
            self._d2 = self.sub._tensor.shape[1]
            self._d3 = self.sub._tensor.shape[2]
            self._d4 = self.sub._tensor.shape[3]

        
        scans_list = [self.sub._tensor[:, :, :, i] for i in range(self.d4)] # список тензоров снимков фМРТ
        voxels = [scan.reshape(self._d1 * self._d2 * self._d3).numpy() for scan in scans_list] # список снимков фМРТ, развернутых в векторы
        data = [(vector_list[n], voxels[k]) for n, k in pairs] # (изображение, снимок)

        # train, test
        l = int(self.train_size * self.d4) # размер обучающей выборки
        train, test = data[:l], data[l:]
            
        train = [(pair[0], self.preprocess(pair[1])) for pair in train]
        test = [(pair[0], self.preprocess(pair[1])) for pair in test]

        return train, test

    def get_XY(self):
        X_train = np.array([pair[0] for pair in self.train])
        Y_train = np.array([pair[1] for pair in self.train]).T
        X_test = np.array([pair[0] for pair in self.test])
        Y_test = np.array([pair[1] for pair in self.test]).T
        return X_train, Y_train, X_test, Y_test
    

## Предсказание снимка

In [166]:
class LinearPredictor(Preprocessor):
    
    def __init__(self, sub, dt, coef, alpha, train_size=0.7):
        super().__init__(sub, dt, coef, train_size)
        self.alpha = alpha
        self.output_name = self._get_output_name()

    def _get_output_name(self):
        output_name = f"output-{self.dt}-{self.sub.number}-{self.coef}"
        if (self.alpha > 0):
            output_name = output_name + f"-reg-{self.alpha}.npy"
        
        folder_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "src", 'subs', f"sub-{self.sub.number}")

        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        output_name = os.path.join(folder_path, output_name)

        return output_name

    def _predict(self):
        W = [] # матрица весов модели

        if (self.alpha > 0):
            A = np.linalg.inv(self.X_train.T @ self.X_train + self.alpha * np.identity(self.X_train.shape[1])) @ self.X_train.T
        else:
            A = np.linalg.pinv(self.X_train)
        
        for i in range(self._d1 * self._d2 * self._d3):
            Y_train_vector = self.Y_train[i]
            w = A @ Y_train_vector
            W.append(w)
            
        W = np.array(W) # w будут строками

        Y_train_predicted = W @ self.X_train.T
        Y_test_predicted = W @ self.X_test.T

        MSE_train = self.MSE(Y_train_predicted - self.Y_train)
        MSE_test = self.MSE(Y_test_predicted - self.Y_test)
    
        return W, Y_train_predicted, Y_test_predicted, MSE_train, MSE_test

    def predict(self):
        if not os.path.exists(self.output_name):
            output = np.array(self._predict(), dtype=object)
            self.W, self.Y_train_predicted, self.Y_test_predicted, self.MSE_train, self.MSE_test = output
            np.save(self.output_name, output, allow_pickle=True)
        else:
            output = np.load(self.output_name, allow_pickle=True)
            self.W, self.Y_train_predicted, self.Y_test_predicted, self.MSE_train, self.MSE_test = output

## Предсказание разницы между снимками

In [165]:
class LinearDeltaPredictor(Preprocessor):
    def __init__(self, sub, dt, coef, alpha, train_size=0.7):
        super().__init__(sub, dt, coef, train_size)
        self.alpha = alpha
        self.output_name = self._get_output_name()
        self.deltaY_train, self.deltaY_test = self.get_deltaY()

    def _get_output_name(self):
        output_name = f"output-{self.dt}-{self.sub.number}-{self.coef}"
        if (self.alpha > 0):
            output_name = output_name + f"-reg-{self.alpha}.npy"
        
        folder_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "src", 'subs_delta', f"sub-{self.sub.number}")

        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        output_name = os.path.join(folder_path, output_name)

        return output_name

    def get_deltaY(self):
        delta_train = [(self.train[n][0], self.train[n][1] - self.train[n-1][1]) for n in range(1, len(self.train))]
        delta_test = [(self.test[n][0], self.test[n][1] - self.test[n-1][1]) for n in range(1, len(self.test))]
        deltaY_train = np.array([pair[1] for pair in delta_train]).T
        deltaY_test = np.array([pair[1] for pair in delta_test]).T

        return deltaY_train, deltaY_test

    def _predict(self):
        W = [] # матрица весов модели

        if (self.alpha > 0):
            A = np.linalg.inv(self.X_train.T @ self.X_train + self.alpha * np.identity(self.X_train.shape[1])) @ self.X_train.T
        else:
            A = np.linalg.pinv(self.X_train)
        
        for i in range(self._d1 * self._d2 * self._d3):
            deltaY_train_vector = self.Y_train[i]
            w = A @ deltaY_train_vector
            W.append(w)
            
        W = np.array(W) # w будут строками

        deltaY_train_predicted = W @ self.X_train.T
        deltaY_test_predicted = W @ self.X_test.T
        Y_train_predicted = np.delete(self.Y_train, -1, 1) + deltaY_train_predicted
        Y_test_predicted = np.delete(self.Y_test, -1, 1) + deltaY_test_predicted
    
        MSE_train = self.MSE(Y_train_predicted - np.delete(self.Y_train, 0, 1))
        MSE_test = self.MSE(Y_test_predicted - np.delete(self.Y_test, 0, 1))

        return W, deltaY_train_predicted, Y_train_predicted, deltaY_test_predicted, Y_test_predicted, MSE_train, MSE_test

    def predict(self):
        if not os.path.exists(self.output_name):
            output = np.array(self._predict(), dtype=object)
            self.W, self.deltaY_train_predicted, self.Y_train_predicted, self.deltaY_test_predicted, self.Y_test_predicted, self.MSE_train, self.MSE_test = output
            np.save(self.output_name, output, allow_pickle=True)
        else:
            output = np.load(self.output_name, allow_pickle=True)
            self.W, self.deltaY_train_predicted, self.Y_train_predicted, self.deltaY_test_predicted, self.Y_test_predicted, self.MSE_train, self.MSE_test = output

In [167]:
sub = Sub('04')

In [188]:
delta = LinearDeltaPredictor(sub=sub, dt=4, coef=1, alpha=1000)

In [185]:
linpred.predict()

In [189]:
delta.predict()

MemoryError: Unable to allocate 2.50 GiB for an array with shape (163840, 2048) and data type float64

In [187]:
linpred.MSE_test

153.45747869048895